In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
#%matplotlib notebook
%reload_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize'] = 10, 5
np.set_printoptions(precision=5, suppress=True)

# Toy example

In [ ]:
from trajectory import Trajectory

np.random.seed(1)
t1 = Trajectory(dim=2, n_complexity=2, model='polynomial', 
                coeffs=np.array([[0, 0], [0, 1]]))
t2 = Trajectory(dim=2, n_complexity=2, model='polynomial', 
                coeffs=np.array([[-3, 3], [1, 0]]))
times1 = np.linspace(0, 1, 10)
times2 = np.linspace(1, 2, 20)
anchors = 4 * np.random.rand(2, 5)
fig, ax = plt.subplots()
ax.scatter(*anchors, color='red')
t1.plot(times=times1, ax=ax, color='black')
t2.plot(times=times2, ax=ax, color='black')
ax.axis('equal')

from measurements import get_measurements

b1, D1 = get_measurements(t1, anchors, seed=1, times=times1)
b2, D2 = get_measurements(t2, anchors, seed=1, times=times2)
F = np.hstack((b1, b2))
D = np.vstack((D1, D2))
times = np.r_[times1, times2]
print(D.shape, F.shape)

#plt.matshow(D)

In [ ]:
## Convert toy example to something more realistic. 
df = pd.DataFrame(columns=['timestamp', 'anchor_id', 'distance'])
i = 0

anchor_ids = range(anchors.shape[1])
print(anchor_ids)

noise = 1e-3
np.random.seed(1)

for d_mn_row, time in zip(D, times):
    anchor_indexes = np.where(d_mn_row>0)[0]
    for anchor_idx in anchor_indexes:
        distance=d_mn_row[anchor_idx]
        if noise > 0:
            distance += np.random.normal(scale=noise)
        df.loc[i, :] = dict(
            timestamp=time,
            anchor_id=anchor_ids[anchor_idx],
            distance=distance
        )
        i += 1
df.head()

# Averaging Algorithm

Estimate the trajectory recursively over a fixed  time window.

In [ ]:
from solvers import alternativePseudoInverse

def averaging_algorithm(D, basis, times, anchors, t_window=1.0, 
                        plotting=False):
    line_counter = 0

    D_k = np.empty((0, D.shape[1]))
    basis_k = np.empty((basis.shape[0], 0))
    C_list = []
    t_list = np.linspace(0, 2, 20)

    if plotting:
        fig, ax = plt.subplots()
        
    for t_s in t_list:
        tk = times[(times>=t_s) & (times< t_s+t_window)]
        for t_n in tk:
            #d_mn, a_m, t = df.iloc[line_counter, 
            #                       ['distance', 'anchor_id', 'timestamp']]
            # d_mn_row = np.zeros(anchors.shape[1]) # pad with zeros.
            # d_mn_row[anchor_ids==a_m] = d_mn
            # line_counter += 1
            # f_n = t1.get_basis(times=[t]) 

            idx = np.where(times == t_n)[0]

            d_mn_row = D[idx, :]
            f_n = basis[:, idx]

            D_k = np.r_[D_k, d_mn_row]
            basis_k = np.c_[basis_k, f_n]

        try:
            C_k = alternativePseudoInverse(D_k, anchors, basis_k) 
            C_list.append(C_k)

            if plotting:
                trajk = t1.copy()
                trajk.set_coeffs(coeffs=C_k)
                trajk.plot(times=tk, ax=ax)

        except AssertionError:
            print('skipping {:.2f} because only'.format(t_s), np.array(tk))

        D_k = np.empty((0, D.shape[1]))
        basis_k = np.empty((basis.shape[0], 0))
    return C_list, t_list


_ = averaging_algorithm(D, F, times, anchors, t_window=1.0, plotting=True)

# Build-up Algorithm


Refine the current trajectory as long as the new measurements "fit the model" well enough. 

Create a completely new trajectory model with measurements if they stop fitting. 

In [ ]:
def build_up_algorithm(D, basis, anchors, times, eps=1e-3, plotting=False):
    C_k = None
    tk = []

    D_k = np.empty((0, D.shape[1]))
    basis_k = np.empty((basis.shape[0], 0))

    if plotting:
        fig, ax = plt.subplots()

    C_list = []
    t_list = []

    def g():
        r_n = C_k.dot(f_n).reshape((ams.shape[0], 1))
        dist_estimates = np.linalg.norm(ams - r_n, axis=0)
        return np.sum(dist_estimates**2 - d_mn_row)

    for d_mn_row, t_n, f_n in zip(D, times, basis.T):

        d_mn_row = d_mn_row.reshape((1, -1))
        ams = anchors[:, d_mn_row[0]>0]
        ds = d_mn_row[0, d_mn_row[0] > 0]
        if C_k is None or g() <= eps:
            D_k = np.r_[D_k, d_mn_row]
            basis_k = np.c_[basis_k, f_n]
            tk.append(t_n)

            try:
                C_k = alternativePseudoInverse(D_k, anchors, basis_k) 
            except AssertionError:
                print('skipping {:.2f} because only'.format(t_n), np.array(tk))
            #except Exception as e:
            #    raise e

        elif (C_k is not None) and g() > eps:
            print('changing to new trajectory, because error is {:.4f}'.format(g()))

            basis_k = f_n
            D_k = d_mn_row

            C_list.append(C_k)
            t_list.append(tk)


            if plotting:
                trajk = t1.copy()
                trajk.set_coeffs(coeffs=C_k)
                trajk.plot(times=tk, ax=ax)

            tk = [t_n]
            C_k = None

    C_list.append(C_k)
    t_list.append(tk)
    
    if plotting:
        print('print the latest one too.')
        trajk = t1.copy()
        trajk.set_coeffs(coeffs=C_k)
        trajk.plot(times=tk, ax=ax)
    return C_list, t_list

_ = build_up_algorithm(D, F, anchors, times, eps=1e-3, plotting=True)